In [ ]:
from platform import python_version
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, MinMaxScaler

print(f'python {python_version()}')
print(f'pandas {pd.__version__}')
print(f'numpy {np.__version__}')
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from scipy import integrate
import seaborn as sns
print(f'lgb {lgb.__version__}')
import operator
import datetime

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.stats import ks_2samp
import random
import copy

from scipy.stats import norm, kurtosis
from sklearn.metrics import make_scorer
from utils import *

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers, models, optimizers
from keras.layers import Dense, Activation, BatchNormalization, AlphaDropout, Dropout, Add, Concatenate
from keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Lambda, GlobalMaxPooling2D, SeparableConv2D
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Lambda, GlobalMaxPooling1D, SeparableConv1D
from keras.regularizers import l2
from keras.models import Sequential, Model, Input, load_model

from keras.wrappers.scikit_learn import KerasRegressor


In [ ]:
def mish(x):
    return x*K.tanh(K.softplus(x))

def step_decay(epoch):
    initial_lrate = 0.002
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    
    lrate = max(5e-5, lrate)
    return lrate

es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = keras.callbacks.LearningRateScheduler(step_decay, verbose=2)

In [ ]:
import librosa

In [ ]:
train_features = pd.read_csv('./data/train_features.csv')
train_target = pd.read_csv('./data/train_target.csv')
test_features = pd.read_csv('./data/test_features.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')


In [ ]:
train_X = train_features.iloc[:,2:].values.reshape((2800,375,4, 1))
test_X = test_features.iloc[:,2:].values.reshape((700,375,4, 1))
train_y = train_target.iloc[:,1:].values

s1 = train_X.squeeze()[:,:,0]
s2 = train_X.squeeze()[:,:,1]
s3 = train_X.squeeze()[:,:,2]
s4 = train_X.squeeze()[:,:,3]

In [ ]:
n_mels = 128
S1 = list(map(lambda x: librosa.feature.melspectrogram(x, sr=375, n_mels=n_mels, hop_length=160), s1))
S2 = list(map(lambda x: librosa.feature.melspectrogram(x, sr=375, n_mels=n_mels, hop_length=160), s2))
S3 = list(map(lambda x: librosa.feature.melspectrogram(x, sr=375, n_mels=n_mels, hop_length=160), s3))
S4 = list(map(lambda x: librosa.feature.melspectrogram(x, sr=375, n_mels=n_mels, hop_length=160), s4))

In [ ]:
mel1 = np.array(list(map(lambda x: librosa.power_to_db(x, ref=np.max), S1)))
mel2 = np.array(list(map(lambda x: librosa.power_to_db(x, ref=np.max), S2)))
mel3 = np.array(list(map(lambda x: librosa.power_to_db(x, ref=np.max), S3)))
mel4 = np.array(list(map(lambda x: librosa.power_to_db(x, ref=np.max), S4)))

In [ ]:
mel1 /= mel1.min()
mel2 /= mel2.min()
mel3 /= mel3.min()
mel4 /= mel4.min()

In [ ]:
train_X = np.dstack([mel1, mel2, mel3, mel4])

In [ ]:
tr_X, te_X,tr_y, te_y = train_test_split(train_X, train_y, test_size=0.3, random_state=42)

In [ ]:
mm1 = np.expand_dims(mel1, -1)
mm2 = np.expand_dims(mel2, -1)
mm3 = np.expand_dims(mel3, -1)
mm4 = np.expand_dims(mel4, -1)

In [ ]:
train_X = np.dstack([mm1, mm2, mm3, mm4])

In [ ]:
# train_X = train_features.iloc[:,2:].values.reshape((2800,375,4, 1))
# test_X = test_features.iloc[:,2:].values.reshape((700,375,4, 1))
# train_y = train_target.iloc[:,1:].values

# tr_X, te_X,tr_y, te_y = train_test_split(train_X, train_y, test_size=0.3, random_state=42)

In [ ]:
test_X /= np.abs(train_X).max()
train_X /= np.abs(train_X).max()

tr_X, te_X,tr_y, te_y = train_test_split(train_X, train_y, test_size=0.3, random_state=42)

In [ ]:
weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])

def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))


def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04

def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)

In [ ]:
K.clear_session()
def set_model(train_target, shape):  # 0:x,y, 1:m, 2:v
    
    activation = tf.nn.swish
    padding = 'same'
    model = Sequential()
    kernel_initializer = 'he_normal'
    nf = 64
    fs = 2
    ps = 1
    strides = 2

    model.add(Conv2D(nf,fs, padding=padding,input_shape=shape, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))

    model.add(Conv2D(nf,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))

    model.add(Conv2D(nf*2,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))

    model.add(Conv2D(nf*2,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))

    model.add(Conv2D(nf*4,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))

    model.add(Conv2D(nf*4,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))
    
    model.add(Conv2D(nf*8,fs, padding=padding, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=ps, strides=strides))
 
    model.add(Flatten())
    
    model.add(Dense(1024, activation =activation, kernel_initializer=kernel_initializer))
    model.add(Dense(512, activation =activation, kernel_initializer=kernel_initializer))
    model.add(Dense(256, activation =activation, kernel_initializer=kernel_initializer))
    model.add(Dense(128, activation =activation, kernel_initializer=kernel_initializer))
    
    model.add(Dropout(0.5))
    model.add(Dense(4, kernel_initializer=kernel_initializer))

    optimizer = keras.optimizers.Adam()

    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1,
                  optimizer=optimizer,
                 )
    else:
        model.compile(loss=my_loss_E2,
                  optimizer=optimizer,
                 )
        
#     model.summary()

    return model

In [ ]:
nn = set_model(0, train_X.shape[1:])
nn.summary()

In [ ]:
def train(model,X,Y):
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):

    history = model.fit(X, Y,
                  epochs=500,
                  batch_size=32,
                  shuffle=True,
                  validation_split=0.2,
                  verbose = 1,
                  callbacks=[es, lrs])

    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    

    return model

In [ ]:
MODEL_SAVE_FOLDER_PATH = './model/'
sub = pd.read_csv('./data/sample_submission.csv')
te_ys = pd.DataFrame(np.zeros_like(te_y), columns=['X', 'Y', 'M', 'V'])

models = [None for _ in range(3)]
for train_target in range(3):
    model = set_model(train_target, shape=train_X.shape[1:])
#     model = build_model(train_target)
    models[train_target] = train(model,tr_X, tr_y)    
#     best_model = load_best_model(train_target)

#     pred_data_test = models[train_target].predict(test_X)
    val_pred =  models[train_target].predict(te_X)

    
    if train_target == 0: # x,y 학습
        sub.iloc[:,1] = pred_data_test[:,0]
        sub.iloc[:,2] = pred_data_test[:,1]
        te_ys.iloc[:,0] = val_pred[:,0]
        te_ys.iloc[:,1] = val_pred[:,1]

    elif train_target == 1: # m 학습
        sub.iloc[:,3] = pred_data_test[:,2]
        te_ys.iloc[:,2] = val_pred[:,2]

    elif train_target == 2: # v 학습
        sub.iloc[:,4] = pred_data_test[:,3]
        te_ys.iloc[:,3] = val_pred[:,3]
        
val_score = kaeri_metric(te_y, te_ys)
print(val_score)

In [ ]:
model_path = f'./model/{round(val_score, 5)}'
os.mkdir(model_path)
for i in range(3):
    models[i].save(model_path+f'/model_{i}')

In [ ]:
sub